# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

## TODO 

- CHECK CORRECTNESS!
- Use buffer for creating bosonic matrices

Then I can immediately start repeating exercises of 0+1


## Improvements

- Only make matrix of Bosons tensored with fermions (better PS representation)
- State ordering option for b0-f0-b1-f1-b2-f2 OR b0-b1-b2-f0-f1-f2

## "Parameters"

- Potential
- (NEW) Lattice Spacing & Sites
- (NEW) Boundary conditions
- Bosonic dof cutoff
- (NEW) State order - for efficiencies sake.
- Binary Encoding


## Plan 

- Exact diagonalization of Hamiltonian - spectrum vs "parameters"
- VQE in QISKIT - quantum procedure
- Machine Learning ground state - compare to old studies
- Gate count for trotter steps

In [1]:
import sympy as sp
from ham_to_matrix import *
from constants import *


In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( (-1)**n*sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*(-1)**n*Derivative(V(q_{n}), q_{n}) + (-1)**n*Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [13]:
N=3
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [14]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

-0.5*(-1)**n*Derivative(V(q_{n}), q_{n}) + (-1)**n*Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [15]:
ham=0

for i in range(1,N+1):
    ham+=totHam.subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

def potential(n):
    return 0

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)
    
ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))
ham

(4*\chi^{\dagger}_{0}*\chi_{1} - 4*\chi^{\dagger}_{0}*\chi_{2} + 4*\chi^{\dagger}_{1}*\chi_{0} + 4*\chi^{\dagger}_{1}*\chi_{2} - 4*\chi^{\dagger}_{2}*\chi_{0} + 4*\chi^{\dagger}_{2}*\chi_{1} + 4*p_{0}**2 + 4*p_{1}**2 + 4*p_{2}**2 - q_{0}*q_{1} - q_{0}*q_{2} + 2*q_{0}**2 - q_{1}*q_{0} - q_{1}*q_{2} + 2*q_{1}**2 - q_{2}*q_{0} - q_{2}*q_{1} + 2*q_{2}**2)/8

In [16]:
#ham=0
#totHam=bosonNI+bosonI+fermionNI+fermionI
#for i in range(0,N):
#    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

#def potential(n):
#    return q[n]

#for n in range(0,N):
#    ham=ham.subs(V(q[n]),potential(n))
#ham=ham.subs(aLat,aVal).simplify().expand()
#ham

In [17]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

\chi^{\dagger}_{0}*\chi_{1}/2 - \chi^{\dagger}_{0}*\chi_{2}/2 + \chi^{\dagger}_{1}*\chi_{0}/2 + \chi^{\dagger}_{1}*\chi_{2}/2 - \chi^{\dagger}_{2}*\chi_{0}/2 + \chi^{\dagger}_{2}*\chi_{1}/2 - a^{\dagger}_{0}*a^{\dagger}_{1}/16 - a^{\dagger}_{0}*a^{\dagger}_{2}/16 + 3*a^{\dagger}_{0}*a_{0}/8 - a^{\dagger}_{0}*a_{1}/16 - a^{\dagger}_{0}*a_{2}/16 - a^{\dagger}_{0}**2/8 - a^{\dagger}_{1}*a^{\dagger}_{0}/16 - a^{\dagger}_{1}*a^{\dagger}_{2}/16 - a^{\dagger}_{1}*a_{0}/16 + 3*a^{\dagger}_{1}*a_{1}/8 - a^{\dagger}_{1}*a_{2}/16 - a^{\dagger}_{1}**2/8 - a^{\dagger}_{2}*a^{\dagger}_{0}/16 - a^{\dagger}_{2}*a^{\dagger}_{1}/16 - a^{\dagger}_{2}*a_{0}/16 - a^{\dagger}_{2}*a_{1}/16 + 3*a^{\dagger}_{2}*a_{2}/8 - a^{\dagger}_{2}**2/8 + 3*a_{0}*a^{\dagger}_{0}/8 - a_{0}*a^{\dagger}_{1}/16 - a_{0}*a^{\dagger}_{2}/16 - a_{0}*a_{1}/16 - a_{0}*a_{2}/16 - a_{0}**2/8 - a_{1}*a^{\dagger}_{0}/16 + 3*a_{1}*a^{\dagger}_{1}/8 - a_{1}*a^{\dagger}_{2}/16 - a_{1}*a_{0}/16 - a_{1}*a_{2}/16 - a_{1}**2/8 - a_{2}*a^{\dag

In [18]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

-a^{\dagger}_{0}**2/8 to matrix timer took: 44.1190 seconds
-a^{\dagger}_{1}**2/8 to matrix timer took: 23.9853 seconds
-a^{\dagger}_{2}**2/8 to matrix timer took: 23.7193 seconds
-a_{0}**2/8 to matrix timer took: 23.7472 seconds
-a_{1}**2/8 to matrix timer took: 23.6457 seconds
-a_{2}**2/8 to matrix timer took: 23.9643 seconds
\chi^{\dagger}_{0}*\chi_{1}/2 to matrix timer took: 33.4708 seconds
\chi^{\dagger}_{1}*\chi_{0}/2 to matrix timer took: 38.7418 seconds
\chi^{\dagger}_{1}*\chi_{2}/2 to matrix timer took: 34.9559 seconds
\chi^{\dagger}_{2}*\chi_{1}/2 to matrix timer took: 39.4926 seconds
-\chi^{\dagger}_{0}*\chi_{2}/2 to matrix timer took: 32.9233 seconds
-\chi^{\dagger}_{2}*\chi_{0}/2 to matrix timer took: 32.8687 seconds
-a^{\dagger}_{0}*a^{\dagger}_{1}/16 to matrix timer took: 33.5585 seconds
-a^{\dagger}_{0}*a^{\dagger}_{2}/16 to matrix timer took: 33.7216 seconds
-a^{\dagger}_{0}*a_{1}/16 to matrix timer took: 40.5345 seconds
-a^{\dagger}_{0}*a_{2}/16 to matrix timer took: 

In [19]:
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

print(np.isreal(ens).all())

ens

True


array([-0.13466929-0.j, -0.13466929-0.j, -0.08177224-0.j, -0.08177224-0.j,
        0.20834003-0.j,  0.20834003+0.j,  0.4751847 -0.j,  0.4751847 -0.j,
        0.4751847 +0.j,  0.4751847 +0.j,  0.59083253+0.j,  0.59083253+0.j,
        0.69469106-0.j,  0.69469106-0.j,  0.69469106+0.j,  0.69469106+0.j,
        0.8653307 -0.j,  0.8653307 +0.j,  0.91822773-0.j,  0.91822773+0.j,
        1.0449319 -0.j,  1.0449319 +0.j,  1.1453074 -0.j,  1.1453074 +0.j,
        1.1814471 -0.j,  1.1814471 -0.j,  1.1814471 +0.j,  1.1814471 +0.j,
        1.20834   +0.j,  1.20834   +0.j,  1.3267992 -0.j,  1.3267992 -0.j,
        1.3267992 +0.j,  1.3267992 +0.j,  1.3393167 -0.j,  1.3393167 -0.j,
        1.3393167 +0.j,  1.3393167 +0.j,  1.3653307 -0.j,  1.3653307 -0.j,
        1.3653307 -0.j,  1.3653307 +0.j,  1.4182278 -0.j,  1.4182278 +0.j,
        1.4182278 +0.j,  1.4182278 +0.j,  1.4751848 -0.j,  1.4751848 -0.j,
        1.4751848 +0.j,  1.4751848 +0.j,  1.5307605 -0.j,  1.5307605 +0.j,
        1.5807837 -0.j,  

In [21]:
xMat()

array([[0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j]])

In [20]:
xDagMat()

array([[0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j]])

In [11]:
from timer import Timer

term=hoHam.args[0]
cutoff=4
timer=Timer('full term')
timer.start()
hamMat=convert_term_to_matrix(term,cutoff,N,aops,adags,xs,xdags)
timer.stop()

(512, 512)
setup timer took: 0.0149 seconds
a^{\dagger}_{0}**2setup timer took: 0.0005 seconds
a^{\dagger}_{0}**2full timer took: 9.9681 seconds
(512, 512) (512, 512)
a^{\dagger}_{0}**2multiply timer took: 0.0055 seconds
full term timer took: 23.4207 seconds


In [18]:
for size in [4,8,16,32,64,128,256,512]:
    a=np.random.rand(size,size).astype(np.complex64)
    b=np.random.rand(size,size).astype(np.complex64)
    print(a.shape,b.shape)
    timer=Timer(str(size))
    timer.start()
    res=np.matmul(a,b)
    print(np.trace(res))
    timer.stop()

(4, 4) (4, 4)
(3.3710556+0j)
4 timer took: 0.0352 seconds
(8, 8) (8, 8)
(17.694397+0j)
8 timer took: 0.0000 seconds
(16, 16) (16, 16)
(64.37317+0j)
16 timer took: 0.0000 seconds
(32, 32) (32, 32)
(254.13129+0j)
32 timer took: 0.0003 seconds
(64, 64) (64, 64)
(1039.4939+0j)
64 timer took: 0.0025 seconds
(128, 128) (128, 128)
(4102.461+0j)
128 timer took: 0.0011 seconds
(256, 256) (256, 256)
(16422.25+0j)
256 timer took: 0.0045 seconds
(512, 512) (512, 512)
(65657.64+0j)
512 timer took: 0.0101 seconds


In [15]:
for size in [4,8,16,32,64,128,256,512]:
    a=np.random.rand(size,size)
    b=np.random.rand(size,size)
    print(a.shape,b.shape)
    timer=Timer(str(size))
    timer.start()
    res=np.matmul(a,b)
    timer.stop()

(4, 4) (4, 4)
4 timer took: 0.0000 seconds
(8, 8) (8, 8)
8 timer took: 0.0000 seconds
(16, 16) (16, 16)
16 timer took: 0.0000 seconds
(32, 32) (32, 32)
32 timer took: 0.0000 seconds
(64, 64) (64, 64)
64 timer took: 0.0003 seconds
(128, 128) (128, 128)
128 timer took: 0.0002 seconds
(256, 256) (256, 256)
256 timer took: 0.0007 seconds
(512, 512) (512, 512)
512 timer took: 0.0058 seconds


In [ ]:
hamMat

In [ ]:
type(hamMat[2,0])

In [ ]:
np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])

In [ ]:
from matrix_to_ps import *
from binary_encodings import *

In [ ]:
hamMat.shape

In [ ]:
hamPS=matrix_to_pauli_strings(hamMat, standard_encode)

In [ ]:
hamPS=sp.nsimplify(hamPS, tolerance=1e-10)

In [ ]:
#sp.simplify(sp.expand(hamPS))